In [1]:
import pandas as pd
# removed this from wiglecsv: WigleWifi-1.4,appRelease=Kismet202208R1,model=Kismet,release=2022.08.R1,device=kismet,display=kismet,board=kismet,brand=kismet
# csvd = pd.read_csv('/home/vlaghe/proj/warpi/data/buc2run/buc2runweli-20230114-14-53-28-1.wiglecsv')
csvd = pd.read_csv('/path/to/your/file.wiglecsv')
#csvd.describe()  # summarry of data
#csvd.AuthMode.unique() # get uniq vals 4 one column
#csvd.drop_duplicates(subset='MAC') # drop duplicates based on col
csvd.sample(10)  # show 10 random sameples

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/your/file.wiglecsv'